# modify example

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path

my_happy_flow_path = str(Path('../../src').resolve())
my_lib_path = str(Path('my_lib').resolve())

if my_lib_path not in sys.path:
    sys.path.append(my_lib_path)


if my_happy_flow_path not in sys.path:
    sys.path.append(my_happy_flow_path)

## integer example

In Python code, 1/3 would normally be evaluated to a floating-point number, that can never be exactly one third. Mathematical software, like SymPy or Sage, often wants to use exact fractions instead. One way to make 1/3 produce an exact fraction is to wrap the integer literals 1 and 3 in a class:

In [2]:
import ast
import ast_utils

from fractions import Fraction

class IntegerWrapper(ast.NodeTransformer):
    def visit_Num(self, node):
        """
        Wraps all integers in a call to Integer()
        """
        if isinstance(node.n, int):
            return ast.Call(
                func=ast.Name(id='Integer', ctx=ast.Load()),
                args=[node],
                keywords=[]
            )
        return node
    
tree = ast.parse("1/3")
tree = IntegerWrapper().visit(tree)
# Add lineno & col_offset to the nodes we create
ast.fix_missing_locations(tree)
print(ast_utils.unparse(tree))


(Integer(1) / Integer(3))



## plus to mulitply

In [3]:
import ast
import ast_utils

class CrazyTransformer(ast.NodeTransformer):
    def visit_BinOp(self, node):
        node.op = ast.Mult()
        return node
    
source_code = """
def add(arg1, arg2): 
    return arg1 + arg2
""".strip()
    
tree = ast.parse(source_code)

exec(compile(tree, filename="<ast>", mode="exec"))
print(add(4, 5))

tree = ast.fix_missing_locations(CrazyTransformer().visit(tree))
print(ast_utils.unparse(tree))

exec(compile(tree, filename="<ast>", mode="exec"))

print(add(4, 5))

9


def add(arg1, arg2):
    return (arg1 * arg2)

20


## assign pi value

In [4]:
import ast
import ast_utils

class AssignPiValue(ast.NodeTransformer):
    def visit_Name(self, node: ast.Name):
        if node.id == 'pi':
            return ast.Num(n=3.14159265)
        return node

source_code = """
y = 2 * pi
""".strip()
    
tree = ast.parse(source_code)

print(ast_utils.dump_json(tree))

tree = ast.fix_missing_locations(AssignPiValue().visit(tree))
print(ast_utils.unparse(tree))

exec(compile(tree, filename="<ast>", mode="exec"))
print(y)

{
 "_PyType": "Module",
 "body": [
  {
   "_PyType": "Assign",
   "targets": [
    {
     "_PyType": "Name",
     "id": "y",
     "ctx": {
      "_PyType": "Store"
     }
    }
   ],
   "value": {
    "_PyType": "BinOp",
    "left": {
     "_PyType": "Num",
     "n": 2
    },
    "op": {
     "_PyType": "Mult"
    },
    "right": {
     "_PyType": "Name",
     "id": "pi",
     "ctx": {
      "_PyType": "Load"
     }
    }
   }
  }
 ]
}

y = (2 * 3.14159265)

6.2831853


## call function

In [5]:
import ast
import ast_utils

class StringWrapper(ast.NodeTransformer):
    def visit_Str(self, node):
        return ast.Call(
            func=ast.Name(id='wrap_string', ctx=ast.Load()),
            args=[node],
            keywords=[]
        )
    
def wrap_string(s):
    return 'START ' + s + ' END'

source_code = """
print('test string')
""".strip()
    
tree = ast.parse(source_code)

print(ast_utils.dump_json(tree))

tree = ast.fix_missing_locations(StringWrapper().visit(tree))
print(ast_utils.unparse(tree))

exec(compile(tree, filename="<ast>", mode="exec"))

{
 "_PyType": "Module",
 "body": [
  {
   "_PyType": "Expr",
   "value": {
    "_PyType": "Call",
    "func": {
     "_PyType": "Name",
     "id": "print",
     "ctx": {
      "_PyType": "Load"
     }
    },
    "args": [
     {
      "_PyType": "Str",
      "s": "test string"
     }
    ],
    "keywords": [

    ]
   }
  }
 ]
}

print(wrap_string('test string'))

START test string END


## inject code for function

In [20]:
import ast
import ast_utils

class InjectCodeForFunction(ast.NodeTransformer):
    
    def __init__(self, 
                 func_start_code='',
                 func_end_code='',
                 func_root_start_code='', 
                 func_root_end_code='',
                 func_root_name='run'):
        self.func_start_code = func_start_code
        self.func_end_code = func_end_code
        
        self.func_root_start_code = func_root_start_code
        self.func_root_end_code = func_root_end_code
        
        self.func_root_name = func_root_name
        
        
    
    def visit_FunctionDef(self, node):
        if node.name == self.func_root_name:
            node.body.insert(0, ast.parse(self.func_root_start_code))
            node.body.append(ast.parse(self.func_root_end_code))
        else:
            node.body.insert(
                0, 
                ast.parse(
                    self.func_start_code.format(func_name=node.name)
                )
            )
            node.body.append(
                ast.parse(
                    self.func_end_code
                )
            )
            
        self.generic_visit(node)
        return node
        
    
    def visit_AsyncFunctionDef(self, node):
        if node.name == self.func_root_name:
            node.body.insert(0, ast.parse(self.func_root_start_code))
            node.body.append(ast.parse(self.func_root_end_code))
        else:
            node.body.insert(0, ast.parse(self.func_start_code))
            node.body.append(ast.parse(self.func_end_code))
            
        self.generic_visit(node)
        return node
        
    


source_code = """
async def async_print():
    print('async me')

def run():
    def print_me():
        print('me')


    def print_me_002():
        print('me_002')
        
    print('me all')
    print_me()
    print_me_002()
    
    
run()


""".strip()
    
tree = ast.parse(source_code)

print(ast_utils.dump_json(tree))

func_start_code = """
print("{func_name} start_001")
print("{func_name} start_002")
""".strip()

func_end_code = """
print("end_001")
print("end_002") 
""".strip()

func_root_start_code = """
print("root start_001")
print("root start_002")
""".strip()

func_root_end_code = """
print("root end_001")
print("root end_002") 
""".strip()

injectCode = InjectCodeForFunction(
    func_start_code=func_start_code,
    func_end_code=func_end_code,
    func_root_start_code=func_root_start_code,
    func_root_end_code=func_root_end_code
)

tree = ast.fix_missing_locations(injectCode.visit(tree))

changed_source_code = ast_utils.unparse(tree)
print(changed_source_code)

tree = ast.parse(changed_source_code)


exec(compile(tree, filename="<ast>", mode="exec"))

{
 "_PyType": "Module",
 "body": [
  {
   "_PyType": "AsyncFunctionDef",
   "name": "async_print",
   "args": {
    "_PyType": "arguments",
    "args": [

    ],
    "vararg": null,
    "kwonlyargs": [

    ],
    "kw_defaults": [

    ],
    "kwarg": null,
    "defaults": [

    ]
   },
   "body": [
    {
     "_PyType": "Expr",
     "value": {
      "_PyType": "Call",
      "func": {
       "_PyType": "Name",
       "id": "print",
       "ctx": {
        "_PyType": "Load"
       }
      },
      "args": [
       {
        "_PyType": "Str",
        "s": "async me"
       }
      ],
      "keywords": [

      ]
     }
    }
   ],
   "decorator_list": [

   ],
   "returns": null
  },
  {
   "_PyType": "FunctionDef",
   "name": "run",
   "args": {
    "_PyType": "arguments",
    "args": [

    ],
    "vararg": null,
    "kwonlyargs": [

    ],
    "kw_defaults": [

    ],
    "kwarg": null,
    "defaults": [

    ]
   },
   "body": [
    {
     "_PyType": "FunctionDef",
     "name": "pri